<a href="https://colab.research.google.com/github/arthur-siqueira/interpolacao-sistemas-lineares/blob/main/interpolacao_sistemas_lineares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np


x_pontos = np.array([0, 1, 2])
y_pontos = np.array([1, 3, 2])

n = len(x_pontos)
A = np.zeros((n, n))
for i in range(n):
  for j in range(n):
    A[i, j] = x_pontos[i]**j